# CRIM Melodic nGram Maps

## What Can You Do with this Notebook?

This NB explains the Melodic nGram Maps, through which you can display all of the nGrams in one or more pieces.

The method takes two dataframes as basic inputs:  a df of ngrams (all of then, or just the entries, if you prefer), and a df of the total durations of those ngrams (if none is specified, you will just have a small line for the start of each ngram instead of a color block spanning its total duration)


```
display(viz.plot_ngrams_heatmap(ngrams_df, 
                                ngrams_duration=None,  
                                selected_patterns=[], 
                                voices=[], 
                                heatmap_width=800,
                                heatmap_height=300, 
                                includeCount=False))
```



In [1]:
# python imports
from intervals import * 
from intervals import importScore 
from intervals import main_objs
from IPython.display import SVG
from ipywidgets import interact
import altair as alt
import glob as glob
import intervals
import intervals.visualizations as viz
import numpy as np
import os
import pandas as pd
import re
import requests

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")
    

saved_csv folder already exists.
Music_Files folder already exists.


In [3]:
piece_list = ['CRIM_Model_0032.mei',
                     'CRIM_Mass_0025_1.mei',
                     'CRIM_Mass_0025_2.mei',
                     'CRIM_Mass_0025_3.mei',
                     'CRIM_Mass_0025_4.mei',
                     'CRIM_Mass_0025_5.mei']

In [8]:
piece_list_2 = ['https://raw.githubusercontent.com/Elenacrod/Buch1/main/Gloriose_confesor_Guerrero.xml',
                     'https://crimproject.org/mei/CRIM_Mass_0001_2.mei'
                     ]
piece_list_2[0]

'https://raw.githubusercontent.com/Elenacrod/Buch1/main/Gloriose_confesor_Guerrero.xml'

In [4]:
for piece in piece_list:
    prefix = 'https://crimproject.org/mei/' 
# prefix = 'Music_Files/'
    mei_file = piece
    url = prefix + mei_file
    piece = importScore(url)
    

In [5]:
# settings for ngrams, unisons and kind

n=4
combineUnisons=False
kind='d'

## an initial view of the Model, with all entries

In [9]:


# select the model from the list
# model = piece_list[0]
# prefix = 'https://crimproject.org/mei/' 
# # prefix = 'Music_Files/'
# url = prefix + model
model = importScore(piece_list_2[0])

# find entries for model
nr = model.notes(combineUnisons=combineUnisons)
mel = model.melodic(df=nr, kind=kind, compound=False, unit=0, end=False)
mod_mel_ngrams = model.ngrams(df=mel, n=n)

# pass the following ngrams to the plot below as first df
mod_entry_ngrams = model.entries(df=mel, n=n, thematic=True, anywhere=True)

# pass the ngram durations below to the plot as second df
mod_mel_ngrams_duration = model.durations(df=mel, n=n, mask_df=mod_entry_ngrams)
mod_entries_stack = list(mod_entry_ngrams.stack().unique())

print(model.metadata)

display(viz.plot_ngrams_heatmap(mod_entry_ngrams, 
                                mod_mel_ngrams_duration, 
                                selected_patterns=[], 
                                voices=[], 
                                heatmap_width=800,
                                heatmap_height=300))

{'title': 'Gloriose confesor Domini', 'composer': 'Francisco Guerrero\nMaría Elena Cuenca (ed.)', 'date': None}


alt.VConcatChart(...)

## Now the Model Paired with Each Mass Movement

Showing only the 'shared' entries in each pair

In [8]:
# set parameters:

n=4
combineUnisons=False
kind='d'

# select the model from the list
model = piece_list[0]
prefix = 'https://crimproject.org/mei/' 
# prefix = 'Music_Files/'
url = prefix + model
model = importScore(url)

# find entries for model
nr = model.notes(combineUnisons=combineUnisons)
mel = model.melodic(df=nr, kind=kind, compound=False, unit=0, end=False)
mod_mel_ngrams = model.ngrams(df=mel, n=n)
mod_entry_ngrams = model.entries(df=mel, n=n, thematic=True, anywhere=True)
mod_mel_ngrams_duration = model.durations(df=mel, n=n, mask_df=mod_entry_ngrams)
mod_entries_stack = list(mod_entry_ngrams.stack().unique())

# find entries mass movements:
mass_movements = piece_list[1:6]

for movement in mass_movements:
    
    prefix = 'https://crimproject.org/mei/' 
    url = prefix + movement
    m_movement = importScore(url)
    nr = m_movement.notes(combineUnisons=combineUnisons)
    mel = m_movement.melodic(df=nr, kind=kind, compound=False, unit=0, end=False)
    mass_mvmt_mel_ngrams = m_movement.ngrams(df=mel, n=n)
    mass_mvmt_entries = m_movement.entries(df=mel, n=n, thematic=True, anywhere=True)
    mass_mvmt_mel_ngrams_duration = m_movement.durations(df=mel, n=n, mask_df=mass_mvmt_entries)
    mass_mvmt_entries_stack = mass_mvmt_entries.stack()

    
    # find shared entries
    
    shared_entries = list(set(mass_mvmt_entries_stack).intersection(mod_entries_stack))

    # print model metadata and chart
    print(model.metadata)

    display(viz.plot_ngrams_heatmap(mod_entry_ngrams, mod_mel_ngrams_duration, 
                        selected_patterns=shared_entries,
                        voices=[], includeCount=False))


    # print mass movement metadata and chart
    print(m_movement.metadata)
    display(viz.plot_ngrams_heatmap(mass_mvmt_entries, mass_mvmt_mel_ngrams_duration, 
                                selected_patterns=shared_entries,
                                voices=[], includeCount=True)) 

{'title': 'Sancta et immaculata virginitas', 'composer': 'Cristóbal de Morales', 'date': 1546}


alt.Chart(...)

{'title': 'Missa Sancta et immaculata virginitas: Kyrie', 'composer': 'Francisco Guerrero', 'date': 1566}


alt.VConcatChart(...)

{'title': 'Sancta et immaculata virginitas', 'composer': 'Cristóbal de Morales', 'date': 1546}


...

alt.VConcatChart(...)

{'title': 'Sancta et immaculata virginitas', 'composer': 'Cristóbal de Morales', 'date': 1546}


alt.Chart(...)

{'title': 'Missa Sancta et immaculata virginitas: Agnus Dei', 'composer': 'Francisco Guerrero', 'date': 1566}


alt.VConcatChart(...)